# Actividad 4
## Alumno
Noé Guerrero Ascencio


# Módulos

In [43]:
import panel as pn
import panel.widgets as pnw
#pn.extension("plotly")
import holoviews as hv

pn.extension(sizing_mode = 'stretch_width')

import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from io import BytesIO

# Clases

In [68]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    
    #Set step length
    def setlength(self, value):
        length = self.get_length()
        self.x *= value/length
        self.y *= value/length
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)
    
class DashBoard:
    trayectoria_actual = pd.DataFrame()
    metrica = pd.DataFrame()
    

# Funciones

## Funciones de recorridos aleatorios

In [3]:
def bm_2d(n_steps = 1000, speed = 6, s_pos=[0, 0]):
    """
      Arguments:
        n_steps:
        speed:
        s_pos:
      Returns:
        BM_2d_df
    """
    #n_steps = 1000
    #s_pos = [0,0]
    #speed = 6

    # Init velocity vector
    velocity = Vec2d(speed, 0)

    BM_2d_df = pd.DataFrame(columns = ["x_pos", "y_pos"])
    temp_df = pd.DataFrame([{"x_pos": s_pos[0], "y_pos": s_pos[1]}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps - 1):
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{"x_pos": BM_2d_df.x_pos[i] + velocity.x, "y_pos": BM_2d_df.y_pos[i] + velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
    #return DF
    return BM_2d_df

def CRW_2d(n_steps = 1000, speed = 6, coef = 0.6, s_pos=[2, 5]):
    """
      Arguments:
        n_steps:
        speed:
        coef: El coeficiente de wrapcauchy con el que se desea
        s_pos:
      Returns:
        BM_2d_df
    """
    #n_steps = 1000
    #s_pos = [0,0]
    #speed = 6

    # Init velocity vector
    velocity = Vec2d(speed, 0)

    BM_2d_df = pd.DataFrame(columns = ["x_pos", "y_pos"])
    temp_df = pd.DataFrame([{"x_pos": s_pos[0], "y_pos": s_pos[1]}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps - 1):
        turn_angle = wrapcauchy.rvs(c=coef)
        #print(turn_angle)
        #turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{"x_pos": BM_2d_df.x_pos[i] + velocity.x, "y_pos": BM_2d_df.y_pos[i] + velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
    #return DF
    return BM_2d_df

def Levy_2d(n_steps = 1000, alpha = 1.9, wrapcauchy_exponent = 0.7, speed=3.0, s_pos = [0, 0]):
    

    # Variables para Levy
    Levy_exponent = alpha #Exponente de Levy
    std_motion_steps = speed #Número de pasos promedio que dara nuestro agente
    beta_exp = 0 #Se usa beta = 0 y la gráfica toma una figura de campana de Gauss

    # Init velocity vector
    speed = std_motion_steps
    velocity_vector = Vec2d(speed, 0)

    #times aux
    time = np.linspace(0, 1, n_steps)

    # Init dataframe
    LW_3d_df = pd.DataFrame(columns=["x_pos", "y_pos"])

    # auxiliar para concatenar
    temp_df = pd.DataFrame([{"x_pos":s_pos[0], "y_pos": s_pos[1]}])
    LW_3d_df = pd.concat([LW_3d_df, temp_df], ignore_index=True)

    for i in range(1, n_steps):
        #print(speed)
        turn_angle = wrapcauchy.rvs(wrapcauchy_exponent) #Giramos con la distribución wrapcauchy
        new_speed = levy_stable.rvs(alpha=Levy_exponent, beta=beta_exp, loc=std_motion_steps)
        #Se establece la nueva velocidad o step length
        velocity_vector.setlength(new_speed)
        
        #Luego giramos
        velocity_vector = velocity_vector.rotated(turn_angle)
        
        temp_df = pd.DataFrame([{"x_pos": LW_3d_df.x_pos[i - 1] + velocity_vector.x, "y_pos": LW_3d_df.y_pos[i - 1] + velocity_vector.y}])
        LW_3d_df = pd.concat([LW_3d_df, temp_df], ignore_index=True)

        #Actualizar speed
        speed = new_speed
    
    return LW_3d_df




## Funciones adicionales

In [69]:
def get_distance(vector_1, vector_2):
    """
      Gets the distance between to polar coordenates given by 2 arrays or lists
      Arguments:
        vector_1: A list or array whose first element (position 0) represents the 'x' coordenate and the second (position 1) 'y' 
        vector_2: A list or array whose first element (position 0) represents the 'x' coordenate and the second (position 1) 'y'
      Returns;
        The listance between the given coordinates
    """
    return math.sqrt((vector_1[0] - vector_2[0])**2 + (vector_1[1] - vector_2[1])**2)

def download_trajectory():    
    return BytesIO(DashBoard.trayectoria_actual.to_csv().encode())

def download_metrics():    
    return BytesIO(DashBoard.metrica.to_csv().encode())

In [103]:
css = '''
.centrado{
    text-align: center !important;
}
.bk.elemento-centrado{
    /* Esta clase requiere que el elemento tenga width definido */
    left: 0;
    right: 0;
    margin-left: auto;
    margin-right: auto;
    
}
.bk.botones_tipo_recorrido {
    width: 300px;
    margin: 5px auto;
    text-align: center;
}
.max-width{
    width: 100% !important;
}
.celda{
    border: solid black thin !important;
    border-collapse: collapse !important;
}
.negrita{
    font-weight: bold !important;
}
#sidebar{
    background-color: #d6e6ef;
}

.spinner {
   position: absolute;
   left: 50%;
   top: 50%;
   height:60px;
   width:60px;
   margin:0px auto;
   -webkit-animation: rotation .6s infinite linear;
   -moz-animation: rotation .6s infinite linear;
   -o-animation: rotation .6s infinite linear;
   animation: rotation .6s infinite linear;
   border-left:6px solid rgba(0,174,239,.15);
   border-right:6px solid rgba(0,174,239,.15);
   border-bottom:6px solid rgba(0,174,239,.15);
   border-top:6px solid rgba(0,174,239,.8);
   border-radius:100%;
}

@-webkit-keyframes rotation {
   from {-webkit-transform: rotate(0deg);}
   to {-webkit-transform: rotate(359deg);}
}
@-moz-keyframes rotation {
   from {-moz-transform: rotate(0deg);}
   to {-moz-transform: rotate(359deg);}
}
@-o-keyframes rotation {
   from {-o-transform: rotate(0deg);}
   to {-o-transform: rotate(359deg);}
}
@keyframes rotation {
   from {transform: rotate(0deg);}
   to {transform: rotate(359deg);}
}

'''
pn.extension(raw_css=[css])

# Elementos y Widgets

In [78]:
# Elementos
radio_group_caminata = pnw.RadioButtonGroup(
    name='Tipo Caminata', options=['BM', 'CRW', 'LF'], button_type='success')
radio_group_metrica = pnw.RadioButtonGroup(
    name='Metrica', options=['MSD', 'Path length'], value="Path length", button_type='success')

number_of_steps = pnw.IntSlider(name='Number of steps', start=1, end=1000, step=1, value=300,
                             sizing_mode='fixed', width=350, css_classes=['elemento-centrado'])

start_x = pnw.IntInput(name="x starting pos", value=15, step=1, start=0, end=50, sizing_mode='fixed',
                      width=250)

start_y = pnw.IntInput(name="y starting pos", value=15, step=1, start=0, end=50, sizing_mode='fixed',
                      width=250)

speed = pnw.IntSlider(name='speed', start=1, end=10, step=1, value=3,
                             sizing_mode='fixed', width=350, css_classes=['elemento-centrado'])
#Coeficiente para Wrapcouchy
wrapcauchy_coef = pnw.FloatSlider(name="wrapcauchy coeficient", value=0.6, step=0.1, start=0.1, end=.9,
                                 sizing_mode='fixed', width=350, height=40, css_classes=['elemento-centrado'])

fila_wrapcauchy_coef = pn.Row(wrapcauchy_coef, visible=False, css_classes=['elemento-centrado'])

# Coeficiente para Levy
levy_coef = pnw.FloatSlider(name="Levy alpha coeficient", value=1.2, step=0.1, start=0.1, end=1.9,
                                 sizing_mode='fixed', width=350, height=40, css_classes=['elemento-centrado'])

fila_levy_coef = pn.Row(levy_coef, visible=False, css_classes=['elemento-centrado'])

file_download_walking_csv = pn.widgets.FileDownload(filename=f"trayectoria.csv", callback=download_trajectory, button_type="primary")

file_download_metrics_csv = pn.widgets.FileDownload(filename=f"metrics.csv", callback=download_metrics, button_type="primary")


In [101]:
#Funciones
def graficar_metrica():
    #graficar_metrica.loading_indicator = True
    #fila_metricas.loading_indicator = True
    metrica = radio_group_metrica.value
    trayectoria = DashBoard.trayectoria_actual
    tipo_caminata = radio_group_caminata.value
    if metrica == "MSD":
        line_name = f"MSD for {tipo_caminata} ({trayectoria.shape[0]} steps)"
        MSD_BM = np.empty(shape=(0))
        total = trayectoria.shape[0]
        for tau in range(1,trayectoria.shape[0]):
            ## start - Add your code here
            MSD_tau = np.empty(shape=(0))
            for n in range(0, total - tau, 1):
                vector_n = [trayectoria.iloc[n].x_pos, trayectoria.iloc[n].y_pos]
                vector_tau_mas_n = [trayectoria.iloc[tau + n].x_pos, trayectoria.iloc[tau + n].y_pos]
                MSD_tau = np.append(MSD_tau, (get_distance(vector_n, vector_tau_mas_n)**2))
            #print(len(MSD_tau))
            MSD_BM = np.append(MSD_BM, np.sum(MSD_tau)/len(MSD_tau))
            ## end - Add your code here
        metrica_values = MSD_BM
    
    elif metrica == "Path length":
        line_name = f"Path Length for {tipo_caminata} ({trayectoria.shape[0]} steps)"
        distancias = np.array([get_distance([trayectoria.iloc[i -1].x_pos, trayectoria.iloc[i -1].y_pos],
                                                      [trayectoria.iloc[i].x_pos, trayectoria.iloc[i].y_pos])
                                         for i in range(1, trayectoria.shape[0])])
        metrica_values = np.cumsum(distancias)
    
    DashBoard.metrica = pd.DataFrame(metrica_values, columns=[metrica])
    fig_metrica = go.Figure()

    # First trace BM1
    ## start - Add your code here
    fig_metrica.add_trace(go.Scatter(
        x = np.arange(len(metrica_values)),
        y = metrica_values,
        marker = dict(size=2),
        line = dict(width=3),
        mode = 'lines',
        name = line_name,
        showlegend = True
    ))
    #fila_metricas.loading_indicator = False
    return fig_metrica
        

@pn.depends(radio_group_caminata, start_x, start_y, speed, number_of_steps, wrapcauchy_coef, levy_coef)
def graficar_trayectoria(radio_group_caminata, start_x, start_y, speed, number_of_steps, wrapcauchy_coef, levy_coef):
    # Obtener parametros
    tipo_trajectoria = radio_group_caminata
    starting_position = [start_x, start_y]
    #n_steps = number_of_steps.value
    #speed_val = speed.value
    
    # Get trajectory (must be a DataFrame with x_pos and y_pos)
    if tipo_trajectoria == "BM":
        title = f"Brownian Motion {number_of_steps} steps"
        line = f"BM, {number_of_steps} steps;"
        trajectory = bm_2d(n_steps=number_of_steps, speed=speed, s_pos=starting_position)
        fila_wrapcauchy_coef.visible = False
        fila_levy_coef.visible = False
    elif tipo_trajectoria == "CRW":
        title = f"Correlated Random Walk {number_of_steps} steps"
        line = f"CRW, coef={round(wrapcauchy_coef,2)};"
        trajectory = CRW_2d(n_steps = number_of_steps, speed = speed, coef = wrapcauchy_coef, s_pos=starting_position)
        fila_wrapcauchy_coef.visible = True
        fila_levy_coef.visible = False
    elif tipo_trajectoria == "LF":
        title = f"Levy Flight {number_of_steps} steps"
        line = f"LF, coef={round(wrapcauchy_coef, 2)}; alpha={round(levy_coef, 2)};"
        trajectory = Levy_2d(n_steps = number_of_steps, alpha = levy_coef, wrapcauchy_exponent = wrapcauchy_coef,
                             speed=3.0, s_pos = starting_position)
        fila_wrapcauchy_coef.visible = True
        fila_levy_coef.visible = True
        
    # Se asigna a la variable de la clase
    DashBoard.trayectoria_actual = trajectory
    
    # Se crea la gráfica
    fig_trajectory_rw = go.Figure()
    fig_trajectory_rw.update_layout(
        title_text = title,
        height = 600,
        scene = dict(
            xaxis = dict(title = "x pos"),
            yaxis = dict(title = "y pos"),
            zaxis = dict(title = "time")
        )
    )
    fig_trajectory_rw.add_trace(
        go.Scatter3d(
            x = trajectory.x_pos,
            y = trajectory.y_pos,
            z = trajectory.index,
            marker = {"size": 2},
            line = {"color": "red", "width": 2},
            mode = "lines",
            name = line,
            showlegend = True
        )
    )
    
    return fig_trajectory_rw

@pn.depends(radio_group_caminata, start_x, start_y, speed, number_of_steps, wrapcauchy_coef, levy_coef)
def obtener_informacion(radio_group_caminata, start_x, start_y, speed, number_of_steps, wrapcauchy_coef, levy_coef):
    basic_info = pn.Column(
        "### Random Walk Info",
        pn.Row(
            pn.Column("speed: ", css_classes=["celda", "negrita"]),
            pn.Column(f"{speed}", css_classes=["celda"])
        ),
        pn.Row(
            pn.Column("steps: ", css_classes=["celda", "negrita"]),
            pn.Column(f"{number_of_steps}", css_classes=["celda"])
        ),
        pn.Row(
            pn.Column("start x: ", css_classes=["celda", "negrita"]),
            pn.Column(f"{start_x}", css_classes=["celda"])
        ),
        pn.Row(
            pn.Column("start y: ", css_classes=["celda", "negrita"]),
            pn.Column(f"{start_y}", css_classes=["celda"])
        )
    )
    if radio_group_caminata == "CRW" or radio_group_caminata == "LF":
        basic_info.append(
            pn.Row(
                pn.Column("wrapcauchy: ", css_classes=["celda", "negrita"]),
                pn.Column(f"{wrapcauchy_coef}", css_classes=["celda"])
            )
        )
    if radio_group_caminata == "LF":
        basic_info.append(
            pn.Row(
                pn.Column("levy alpha: ", css_classes=["celda", "negrita"]),
                pn.Column(f"{levy_coef}", css_classes=["celda"])
            )
        )
        
    return basic_info

@pn.depends(radio_group_caminata, start_x, start_y, speed, number_of_steps, wrapcauchy_coef, levy_coef, radio_group_metrica)
def graficar_metrica_bind_function(radio_group_caminata, start_x, start_y, speed, number_of_steps, wrapcauchy_coef, levy_coef, radio_group_metrica):
    return graficar_metrica()

@pn.depends(radio_group_metrica)
def metrics_value(radio_group_metrica):
    return f"Descargar {radio_group_metrica}"

In [210]:
DashBoard.trayectoria_actual

,x_pos,y_pos
0,13,13
1,17.484823,15.210513
2,21.743788,17.829904
3,22.393371,12.87228
4,26.668048,10.278608
...,...,...
216,34.217289,-160.637665
217,37.408135,-156.788184
218,42.338845,-155.958668
219,41.060034,-151.124969


In [104]:
bootstrap = pn.template.BootstrapTemplate(title='My Dashboard - Tópico de Industria I')

bootstrap.sidebar.append(
    pn.Column(
        "# Adicional",
        obtener_informacion,
        "## Descargas",
        "Descargar caminata",
        file_download_walking_csv,
        metrics_value,
        file_download_metrics_csv,
        css_classes=["centrado"]
    )
)

bootstrap.main.append(
    pn.Column(
        "# Elija una caminata",
        pn.Row(
            pn.Spacer(),
            radio_group_caminata,
            pn.Spacer()
        ),
        pn.Row(
            pn.Column(
                "## Controles",
                number_of_steps,
                pn.Row(
                    start_x,
                    start_y,
                    css_classes=['elemento-centrado'],
                ),
                speed,
                fila_wrapcauchy_coef,
                fila_levy_coef
            ),
            pn.Column(
                "## 3D Graph",
                graficar_trayectoria
            )
        ),
        css_classes=['centrado'],
        
    )
)
bootstrap.main.append(
    pn.Row(        
        pn.Column(
            "# Elija una métrica",
            pn.Row(
                pn.Spacer(),
                radio_group_metrica,
                pn.Spacer()
            )
        ),
        
        # Gráfica
        pn.Column(
            graficar_metrica_bind_function
        ),
        css_classes=['centrado']
    )
    
)



bootstrap.show()

Launching server at http://localhost:51601
